# Traitement et stockage des données issues du scraping

Vous avez scrapé les données du site de livres et les avez stockées dans un fichier. 

L'objectif de ce notebook est de créer une base de données pour y stocker ces données.

In [89]:
import sqlite3
import pandas as pd

Lire les données du fichier sauvegardé en utilisant pandas.

In [90]:
# Lire les données du fichier que vous venez d'enregistrer
df_books = pd.read_csv("books_infos.csv")
df_books.head()

,title,price,rating,availability
0,A Light in the Attic,£51.77,Three,In stock
1,Tipping the Velvet,£53.74,One,In stock
2,Soumission,£50.10,One,In stock
3,Sharp Objects,£47.82,Four,In stock
4,Sapiens: A Brief History of Humankind,£54.23,Five,In stock


## 1. Prétraitement des données

On souhaite créer la table _book_ contenant les attributs suivants : 
- id : INT, PK,
- title : TEXT,
- price : DECIMAL
- availability : BOOLEAN
- rating : INT [0:5]

Vérifier les types des colonnes du dataframe.

In [92]:
# Vérification des types de données
df_books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         1000 non-null   object
 1   price         1000 non-null   object
 2   rating        1000 non-null   object
 3   availability  1000 non-null   object
dtypes: object(4)
memory usage: 31.4+ KB


Dans les cellules qui suivent, des méthodes de traitement de données sont suggérées pour donner un aperçu de ce qu'il est possible de faire avec pandas.

**Il est tout à fait possible de faire autrement.**

Utiliser la méthode pandas [_astype_](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html) pour convertir la colonne de titre en chaîne de caractère.

In [59]:
# # Conversion de title en chaîne de caractères
# df_books["title"] = df_books["title"].astype("string")

# # Vérification du type de la colonne title
# print(df_books["title"].dtype)

Pour convertir la colonne de prix en nombre décimal, il est nécessaire d'utiliser une étape intermédiaire pour retirer le caractère "£".

Il est possible par exemple d'utiliser l'attribut [.str](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.html) de la série "price".

In [60]:
# for i in ["$", "€", "£"] : 
#     df_books["price"] = df_books["price"].replace(i, "")
    
# print(df_books["price"].dtype)


In [61]:
# # Convertir la colonne price en type décimal Â
# df_books["price"] = df_books["price"].str.replace("£", " ")
# df_books["price"] = df_books["price"].str.replace("Â", " ")
# df_books["price"] = df_books["price"].str.strip()
# df_books["price"] = df_books["price"].astype(float)
# # Vérification du type de la colonne price

# print(df_books["price"].dtype)


Convertir la colonne `availability` en boolen (True/False).

Quelles sont les valeurs possibles pour la colonne availability ?

In [62]:
# #df_books['availability'] = df_books['availability'].astype("bool")
# print(df_books['availability'])
# print(df_books['availability'].unique())
# print(df_books['availability'].dtype)

Créer une fonction qui prend en entrée la valeur de `availability` et qui renvoie True ou False en fonction de la valeur d'entrée.

In [93]:
# Fonction pour convertir la valeur de availability en booléen
def convert_availability(value : str) -> bool:
    """Convert the availability value to a boolean.

    Args:
        value (str): The availability status of the book.

    Returns:
        bool: True if the book is available, False otherwise.
    """
    if value == "In stock":
        return True
    else :
        return False

Utiliser la méthode [`apply`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html) pour appliquer la fonction à la colonne `availability`.

In [94]:
# Convertir la colonne availability en booléen (True/False)
if df_books['availability'].dtype != 'bool':
    df_books["availability"] = df_books["availability"].apply(convert_availability)

# Vérification du type de la colonne availability
print(df_books["availability"].dtype)
df_books.head() 

bool


,title,price,rating,availability
0,A Light in the Attic,£51.77,Three,True
1,Tipping the Velvet,£53.74,One,True
2,Soumission,£50.10,One,True
3,Sharp Objects,£47.82,Four,True
4,Sapiens: A Brief History of Humankind,£54.23,Five,True


Convertir la colonne _rating_ en chiffre en utilisant un dictionnaire `rating_map` et la méthode [_map_](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.map.html).

In [65]:
# print(df_books["rating"].head().map(repr))
# print(df_books["rating"].unique())


In [66]:
# # Dictionnaire associant les notes au format initial et les valeurs numériques
# ratings_map = {
#     'One': 1,
#     'Two': 2,
#     'Three': 3,
#     'Four': 4,
#     'Five': 5
# }

# df_books["rating"] = df_books["rating"].map(ratings_map)

# # Vérification du type de la colonne rating
# print(df_books["rating"].dtype)

In [95]:
# Créer une fonction convert_types qui combine les traitements faits dans les cellules précédentes
def convert_types(df_books: pd.DataFrame) -> pd.DataFrame:
    """Convert the types of the DataFrame columns to appropriate types.

    Args:
        df_books (pd.DataFrame): The DataFrame containing book data.

    Returns:
        pd.DataFrame: The DataFrame with converted types.
    """
    df = df_books.copy()
    # convert title coluln to str type
    df["title"] = df["title"].astype("string")
    
    # # Convertir la colonne price en type décimal Â
    df_books["price"] = df_books["price"].str.replace("£", " ")
    df_books["price"] = df_books["price"].str.replace("Â", " ")
    df_books["price"] = df_books["price"].str.strip()
        
    # Fonction pour convertir la valeur de availability en booléen
    if df['availability'].dtype != 'bool':
        df["availability"] = df["availability"].apply(convert_availability)
        
        
    ratings_map = {
    'One': 1,
    'Two': 2,
    'Three': 3,
    'Four': 4,
    'Five': 5
}

    df["rating"] = df["rating"].map(ratings_map)
    return df

In [97]:
convert_types(df_books)

,title,price,rating,availability
0,A Light in the Attic,51.77,3,True
1,Tipping the Velvet,53.74,1,True
2,Soumission,50.10,1,True
3,Sharp Objects,47.82,4,True
4,Sapiens: A Brief History of Humankind,54.23,5,True
...,...,...,...,...
995,Our Band Could Be Your Life: Scenes from the A...,57.25,3,True
996,Olio,23.88,1,True
997,Mesaerion: The Best Science Fiction Stories 18...,37.59,1,True
998,Libertarianism for Beginners,51.33,2,True


---
## 2. Insertion des données en base

Dans cette section :
- on créé une BDD sqlite  `book_store.db` (ou on se connecte à la base si elle existe déjà) en utilisant la bibliothèque python sqlite3,
- on insère les données prétraitées dans la BDD

Utiliser le [tutoriel](https://www.ionos.fr/digitalguide/sites-internet/developpement-web/sqlite3-avec-python/) pour l'utilisation de sqlite3.

Utiliser la fonction pandas adaptée qui permet d'insérer un dataframe dans une BDD.

In [98]:
# Création de la BDD et insertion des données
import sqlite3
connection = sqlite3.connect("book_store.db")

In [104]:
print(connection.total_changes)
cursor = connection.cursor()

1000


In [100]:
df_books.to_sql("books_informations", connection, if_exists="append", index=False)

1000

Vérifier le nombre de livres présents dans la BDD en utilisant sqlite3 et la requête SQL adaptée.

In [108]:
# Compter le nombre de livre dans la BDD
cursor.execute("SELECT COUNT(*) FROM books_informations")
books_num = cursor.fetchone()[0]
print(books_num)

1000
